## imports

In [111]:
import pandas as pd
from pathlib import Path
from edisgo import flex_opt
from edisgo import network
from edisgo import tools
from edisgo import EDisGo
from edisgo.network.grids import MVGrid, LVGrid
from edisgo.network.timeseries import get_component_timeseries

# from tabulate import tabulate
# from io import StringIO
# from pypsa import Network as PyPSANetwork

## Some helpful debugging functions

In [112]:

def pd_print(df):
    print(tabulate(df, headers='keys', tablefmt='psql'))
    
# def search_str(df,col,search_str,invert=False):
#     if invert == False:
#         pd_print(df.loc[df[col].str.contains(search_str)])
#         return df.loc[df[col].str.contains(search_str)]
#     else:
#         pd_print(df.loc[~df[col].str.contains(search_str)])
#         return df.loc[~df[col].str.contains(search_str)]
    
def search_str(df,col,search_str,invert=False):
    if invert == False:
        display(df.loc[df[col].str.contains(search_str)])
#         return df.loc[df[col].str.contains(search_str)]
    else:
        display(df.loc[~df[col].str.contains(search_str)])
#         return df.loc[~df[col].str.contains(search_str)]
    
def search_str_df(df,col,search_str,invert=False):
    if invert == False:
        return df.loc[df[col].str.contains(search_str)]
    else:
        return df.loc[~df[col].str.contains(search_str)]

## Get grid dir

In [113]:
grid_name = '1-MVLV-comm-all-0-no_sw'
# grid_name = '1-MVLV-semiurb-3.202-1-no_sw'


curr_path = Path.cwd()
parent_dir = curr_path.parents[3]
# simbench_grids_dir = parent_dir / 'simbench'
grid_dir = parent_dir / grid_name
print (grid_dir)

/Users/heikernleong/Documents/localFiles/sa_code/edisgo/hk_edisgo/1-MVLV-comm-all-0-no_sw


## Importing the simbench raw data into panda dataframes

In [114]:
file_list = [i for i in grid_dir.iterdir()]
simbench_dict = {i.name[:-4]:pd.read_csv(i,delimiter=";") for i in file_list}
# for i in simbench_dict.keys(): print (i)
sb_ding0_dict = {}

## 1) buses_df (Checked for units)

In [115]:
def label_mv_grid_id(name):
    if "LV" in name:
        return ""
    elif "MV" in name:
        return name[0:name.find(" ")]

def label_lv_grid_id(name):
    if "MV" in name:
        return ""
    elif "LV" in name:
        return name[name.find(".")+1:]
    
buses_cols_to_drop = [
    'type',
    'vmSetp',
    'vaSetp',
    'vmMin',
    'vmMax',
    'substation',
    'voltLvl'
]

buses_col_location = [
    'name',
    'x',
    'y',
    'mv_grid_id',
    'lv_grid_id',
    'v_nom',
    'in_building'
]

buses_df = simbench_dict['Node']
buses_df = buses_df.rename(columns={'vmR':'v_nom'})
buses_df = buses_df.rename(columns={'id':'name'})
coord_df = simbench_dict['Coordinates']
coord_df = coord_df.set_index('id')
buses_df['x'] = buses_df['coordID'].apply(lambda coordID: coord_df.loc[coordID,'x'] )
buses_df['y'] = buses_df['coordID'].apply(lambda coordID: coord_df.loc[coordID,'y'] )
buses_df['in_building'] = 'False'

buses_df['mv_grid_id'] =  buses_df['name'].apply(label_mv_grid_id)
buses_df['lv_grid_id'] =  buses_df['subnet'].apply(label_lv_grid_id)

# get the HV grid to adopt the mv grid id
# hv_name = buses_df[buses_df['name'].str.contains('HV')]['name'].iloc[0]
hv_index = buses_df.index[buses_df['name'].str.contains('HV')]
# print (hv_index)
mv_grid_id = buses_df[buses_df['name'].str.contains('MV')]['mv_grid_id'].iloc[0]
buses_df.loc[hv_index,'mv_grid_id'] = mv_grid_id 
# Dropping High Voltage Bus
buses_df = buses_df.drop(hv_index)

buses_df = buses_df[buses_col_location]
display(buses_df.head())
sb_ding0_dict['buses_df'] = buses_df

,name,x,y,mv_grid_id,lv_grid_id,v_nom,in_building
0,LV1.401 Bus 1,11.4545,53.5965,,401,0.4,False
1,LV1.401 Bus 2,11.4537,53.5965,,401,0.4,False
2,LV1.401 Bus 3,11.4520,53.5949,,401,0.4,False
3,LV1.401 Bus 4,11.4533,53.5970,,401,0.4,False
4,LV1.401 Bus 5,11.4528,53.5945,,401,0.4,False


## 6) transformer_df (No direct information on x and r. Left empty for now. Potential source of problem)

In [116]:
trans_rename_dict = {
    'id':'name',
    'nodeHV':'bus0',
    'nodeLV':'bus1',
}

trans_col_location = [
    'name',
    'bus0',
    'bus1',
    's_nom',
    'r',
    'x',
    'type_info'
]

sb_trans_df = simbench_dict['Transformer']
sb_trans_type_df = simbench_dict['TransformerType']
sb_trans_df = sb_trans_df.rename(columns=trans_rename_dict)
sb_trans_type_df = sb_trans_type_df.set_index('id')
sb_trans_df['s_nom'] = sb_trans_df['type'].apply(lambda x: sb_trans_type_df.loc[x,'sR'] )
sb_trans_df['r'] = ''
sb_trans_df['x'] = ''
sb_trans_df['type_info'] = sb_trans_df['type']
sb_trans_df = sb_trans_df[trans_col_location]

transformers_hvmv_df = sb_trans_df[sb_trans_df['bus0'].str.contains('HV')]
transformers_hvmv_df = sb_trans_df[sb_trans_df['bus1'].str.contains('MV')]
transformers_hvmv_df = transformers_hvmv_df.set_index('name')

hvmv_bus_row = transformers_hvmv_df['bus1'][0]
transformers_hvmv_df = transformers_hvmv_df.reset_index()

transformers_df = sb_trans_df[sb_trans_df['bus0'].str.contains('MV')]
transformers_df = sb_trans_df[sb_trans_df['bus1'].str.contains('LV')]
# transformers_df = transformers_df.set_index('name')

display(transformers_hvmv_df.head())
display(transformers_df.head())

sb_ding0_dict['transformers_hvmv_df'] = transformers_hvmv_df
sb_ding0_dict['transformers_df'] = transformers_df

,name,bus0,bus1,s_nom,r,x,type_info
0,HV1-MV4.101-Trafo1,HV1 Bus 13,MV4.101 busbar1.1,40.0,,,40 MVA 110/20 kV YNd5
1,HV1-MV4.101-Trafo2,HV1 Bus 13,MV4.101 busbar1.1,40.0,,,40 MVA 110/20 kV YNd5


,name,bus0,bus1,s_nom,r,x,type_info
0,MV4.101-LV1.401-Trafo 1,MV4.101 Bus 49,LV1.401 Bus 4,0.16,,,0.16 MVA 20/0.4 kV DOTE 160/20 SGB
1,MV4.101-LV2.401-Trafo 1,MV4.101 Bus 8,LV2.401 Bus 19,0.25,,,0.25 MVA 20/0.4 kV Dyn5 ASEA
2,MV4.101-LV2.402-Trafo 1,MV4.101 Bus 10,LV2.402 Bus 19,0.25,,,0.25 MVA 20/0.4 kV Dyn5 ASEA
3,MV4.101-LV2.403-Trafo 1,MV4.101 Bus 11,LV2.403 Bus 19,0.25,,,0.25 MVA 20/0.4 kV Dyn5 ASEA
4,MV4.101-LV2.404-Trafo 1,MV4.101 Bus 17,LV2.404 Bus 19,0.25,,,0.25 MVA 20/0.4 kV Dyn5 ASEA


## 2) generator_df (Checked for units)

In [155]:
gen_cols_to_drop = [
    'pRES',
    'qRES',
    'subnet',
    'voltLvl'
]

gen_col_location = [
    'name',
    'bus',
    'control',
    'p_nom',
    'type',
    'weather_cell_id',
    'subtype'
]

gen_rename_dict = {
    'id':'name',
    'node':'bus',
    'pRES':'p_nom',
    'profile':'weather_cell_id'
}

slack_col_to_drop = [
    'dspf',
    'pExtNet',
    'qExtNet',
    'pWardShunt',
    'qWardShunt',
    'rXWard',
    'xXWard',
    'vmXWard',
    'subnet',
    'voltLvl'
]

def add_slack_terms(gen_df):
    add_gen_dict = {
        'p_nom':0.0,
        'type':'station',
        'weather_cell_id':"",
        'subtype':'mv_station'
    }
    for key in add_gen_dict:
        gen_df[key] = gen_df['control'].apply(lambda x: add_gen_dict[key] if x == 'Slack' else x)
    return gen_df

generators_df = simbench_dict['RES']
generators_df = generators_df.rename(columns=gen_rename_dict)
generators_df['calc_type'] = generators_df['calc_type'].apply(lambda x: x.upper())
generators_df = generators_df.rename(columns={'calc_type':'control'})
generators_df['type'] = generators_df['type'].apply(lambda x: 'solar' if x== 'PV' else x.lower())
generators_df['subtype'] = generators_df['type']
# generators_df = generators_df.drop(gen_cols_to_drop, axis = 1)
generators_df = generators_df[gen_col_location]


# including the slack bus
slack_df = simbench_dict['ExternalNet']
slack_df = slack_df.rename(columns={'id':'name'})
slack_df = slack_df.rename(columns={'node':'bus'})
slack_df = slack_df.rename(columns={'calc_type':'control'})
slack_df = slack_df.drop(slack_col_to_drop,axis=1)
slack_df['control'] = slack_df['control'].apply(lambda x: 'Slack' if x == 'vavm' else x)
slack_df['name'] = slack_df['name'].apply(lambda x: x+'_slack')
slack_df = add_slack_terms(slack_df)
display(slack_df)
slack_df['bus'] = hvmv_bus_row
display(slack_df)
generators_df = pd.concat([slack_df,generators_df] ,ignore_index=True)
# generators_df = generators_df.set_index('name')
display (generators_df.head())

sb_ding0_dict['generators_df'] = generators_df

,name,bus,control,p_nom,type,weather_cell_id,subtype
0,HV1 grid at MV4.101_slack,HV1 Bus 13,Slack,0.0,station,,mv_station


,name,bus,control,p_nom,type,weather_cell_id,subtype
0,HV1 grid at MV4.101_slack,MV4.101 busbar1.1,Slack,0.0,station,,mv_station


,name,bus,control,p_nom,type,weather_cell_id,subtype
0,HV1 grid at MV4.101_slack,MV4.101 busbar1.1,Slack,0.000000,station,,mv_station
1,LV1.401 SGen 1,LV1.401 Bus 7,PQ,0.040000,solar,PV5,solar
2,LV1.401 SGen 2,LV1.401 Bus 11,PQ,0.078381,solar,PV8,solar
3,LV1.401 SGen 4,LV1.401 Bus 8,PQ,0.019000,solar,PV6,solar
4,LV1.401 SGen 3,LV1.401 Bus 13,PQ,0.023000,solar,PV5,solar


## 3) lines_df (Not sure if the s_nom is line to line or 3 phase)

In [118]:
lines_cols_to_drop = [
    'loadingMax',
    'subnet',
    'voltLvl'
]

lines_col_location = [
    'name',
    'bus0',
    'bus1',
    'length',
    'r',
    'x',
    's_nom',
    'num_parallel',
    'kind',
    'type_info'
]

def get_line_char(lines_df, line_type_df):
    char_dict = {
        'r':'r',
        'x': 'x',
        'iMax':'iMax',
        'kind':'type'
    }
    for key in char_dict:
        lines_df[key] = lines_df['type_info'].apply(lambda x: line_type_df.loc[x,char_dict[key]])
    return lines_df

# def cal_s_nom(name):
#     x = name['x']
#     r = name['r']
#     i = name['iMax']
#     return ((((x**2 + r**2))**0.5) * (i)**2)/(10**6)

def cal_s_nom(row):
    i = row['iMax']
    bus0 = row['bus0']
#     print (bus0)
    bus1 = row['bus1']
#     print (bus1)
#     bus0_v_nom = buses_df.loc[bus0,'v_nom']
    bus0_v_nom = search_str_df(buses_df,'name',bus0,invert=False)['v_nom'].values[0]
    bus1_v_nom = search_str_df(buses_df,'name',bus1,invert=False)['v_nom'].values[0]
    if row['bus0'] == 'MV4.101 busbar1.1':
        print('bus0_v_nom: '+ str(bus0_v_nom) + "   " + 'bus1_v_nom: '+ str(bus1_v_nom))
    # display (bus0_v_nom)
#     bus1_v_nom = buses_df.loc[bus1,'v_nom']
    bus_v_nom = (bus0_v_nom+bus1_v_nom)/2 
#     bus_v_nom = search_str_df(buses_df,'name',bus0,invert=False)['v_nom'].values[0]
    return 3**0.5*bus_v_nom*i*10**(-3)
#     return 3*bus_v_nom*i*10**(-3)

# def mul_length(name):
#     return name['x']*name['length']

lines_df = simbench_dict['Line']
line_type_df = simbench_dict['LineType']
line_type_df = line_type_df.set_index('id')

# pd_print(line_type_df.head())
lines_rename_dict = {
    'id':'name',
    'nodeA':'bus0',
    'nodeB':'bus1',
    'type': 'type_info'
}

lines_df = lines_df.rename(columns=lines_rename_dict)
# lines_df = lines_df.drop(lines_cols_to_drop,axis=1)
lines_df = get_line_char(lines_df,line_type_df)
lines_df['kind'] = lines_df['kind'].apply(lambda x : 'line' if x =='ohl' else x)
lines_df['num_parallel'] = 1
lines_df['r'] = lines_df.apply(lambda name: name['r']*name['length'],axis=1)
lines_df['x'] = lines_df.apply(lambda name: name['x']*name['length'],axis=1)
lines_df['s_nom'] = lines_df.apply(cal_s_nom,axis=1)
lines_df = lines_df[lines_col_location]
# lines_df = lines_df.set_index('name')
display(lines_df.head())

sb_ding0_dict['lines_df'] = lines_df

bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0
bus0_v_nom: 20.0   bus1_v_nom: 20.0


,name,bus0,bus1,length,r,x,s_nom,num_parallel,kind,type_info
0,LV1.401 Line 1,LV1.401 Bus 10,LV1.401 Bus 3,0.055767,0.011527,0.004485,0.187061,1,cable,NAYY 4x150SE 0.6/1kV
1,LV1.401 Line 2,LV1.401 Bus 14,LV1.401 Bus 12,0.053576,0.011074,0.004309,0.187061,1,cable,NAYY 4x150SE 0.6/1kV
2,LV1.401 Line 3,LV1.401 Bus 7,LV1.401 Bus 4,0.049814,0.010297,0.004006,0.187061,1,cable,NAYY 4x150SE 0.6/1kV
3,LV1.401 Line 4,LV1.401 Bus 9,LV1.401 Bus 2,0.017886,0.003697,0.001438,0.187061,1,cable,NAYY 4x150SE 0.6/1kV
4,LV1.401 Line 5,LV1.401 Bus 8,LV1.401 Bus 11,0.016089,0.003326,0.001294,0.187061,1,cable,NAYY 4x150SE 0.6/1kV


In [119]:
# display(lines_df)

## 4) loads_df (using of sR as peak load. Need to verify with Anya)

In [120]:
loads_rename_dict = {
    'id':'name',
    'node':'bus',
    'profile': 'sector'
}

loads_cols_to_drop = [
    'pLoad',
    'qLoad',
    'subnet',
    'voltLvl',
    'sR'
]

loads_cols_location = [
    'name',
    'bus',
    'peak_load',
    'annual_consumption',
    'sector'
]

def cal_annual_consumption(name):
    sR = name['sR']
    sector = name['sector']
    loads_profile[sector+'_sload'] = (loads_profile[ sector + '_pload']**2+loads_profile[sector + '_qload']**2)**0.5
    annual_consumption = loads_profile[sector+'_sload'].sum()*sR
    return annual_consumption

def cal_peak_load(name):
    sR = name['sR']
    sector = name['sector']
    peak_load = loads_profile[sector+'_pload'].max()*sR
    return peak_load

loads_profile = simbench_dict['LoadProfile']
loads_df = simbench_dict['Load']
loads_df = loads_df.rename(columns=loads_rename_dict)
loads_df['peak_load'] = loads_df.apply(cal_peak_load,axis=1)
loads_df['annual_consumption'] = loads_df.apply(cal_annual_consumption,axis=1)
loads_df = loads_df[loads_cols_location]
# loads_df = loads_df.set_index('name')
sb_ding0_dict['loads_df'] = loads_df



In [121]:
display(loads_df.sort_values(by=['peak_load'],ascending=False).head())

,name,bus,peak_load,annual_consumption,sector
6850,MV4.101 MV Load 9,MV4.101 Bus 12,1.559140,22692.365756,G4-B
6841,HV1_MV4.101_load,MV4.101 busbar1.1,0.935484,6438.059474,G1-C
6853,MV4.101 MV Load 12,MV4.101 Bus 86,0.569892,7616.076960,G6-A
6856,MV4.101 MV Load 15,MV4.101 Bus 37,0.548387,10557.230935,G3-M
6855,MV4.101 MV Load 14,MV4.101 Bus 36,0.483871,8872.668361,G0-M


## 5) switches_df
currently not implemented due to different means of working on the switches between the 2 systems.

In [122]:
switch_rename_dict = {
    'id':'name',
    'nodeA':'bus_closed',
    'nodeB':'bus_open',
    'substation':'branch',
    'type':'type_info'
}

switch_col_location = [
    'name',
    'bus_closed',
    'bus_open',
    'branch',
    'type_info'
]

switches_df = simbench_dict['Switch']
switches_df = switches_df.rename(columns=switch_rename_dict)
switches_df = switches_df[switch_col_location]
# switches_df = switches_df.set_index('name')

display(switches_df)

sb_ding0_dict['switches_df'] = switches_df

,name,bus_closed,bus_open,branch,type_info
0,MV4.101 BB-RS-line CB Remote Station,MV4.101 RS busbar1A,MV4.101 RS busbar1A_1,MV4.101_Remote_Station,CB
1,MV4.101 loop_line_switch 1.2,MV4.101 Bus 69,MV4.101 Bus 69_1,NaN,LBS
2,MV4.101 loop_line_switch 2.2,MV4.101 Bus 82,MV4.101 Bus 82_1,NaN,LBS
3,MV4.101 loop_line_switch 3.2,MV4.101 Bus 87,MV4.101 Bus 87_1,NaN,LBS
4,MV4.101 loop_line_switch 4.2,MV4.101 Bus 36,MV4.101 Bus 36_1,NaN,LBS
5,MV4.101 RS-Feeder3_line CB Remote Station,MV4.101 RS busbar1A,MV4.101 RS busbar1A_2,MV4.101_Remote_Station,CB
6,MV4.101 RS-Feeder7_line CB Remote Station,MV4.101 RS busbar1A,MV4.101 RS busbar1A_3,MV4.101_Remote_Station,CB


## Storage Unit
Current implementation just includes an empty dataframe (Further implementations will include storage units)

In [123]:
storage_col_dict = {
    'name': pd.Series([],dtype='str'),
    'bus': pd.Series([],dtype='str'),
    'control': pd.Series([],dtype='str'),
    'p_nom': pd.Series([],dtype='float'),
    'capacity': pd.Series([],dtype='float'),
    'efficiency_store': pd.Series([],dtype='float'),
    'efficiency_dispatch': pd.Series([],dtype='float'),
}


storage_units_df = pd.DataFrame(storage_col_dict)
# storage_units_df =storage_units_df.set_index('name')
display(storage_units_df)
sb_ding0_dict['storage_units_df'] = storage_units_df

,name,bus,control,p_nom,capacity,efficiency_store,efficiency_dispatch


## EDisGo direct import

In [124]:
edisgo_obj = EDisGo(import_timeseries=False)
edisgo_obj.topology.buses_df = buses_df.set_index('name')
edisgo_obj.topology.generators_df = generators_df.set_index('name')
edisgo_obj.topology.lines_df = lines_df.set_index('name')
edisgo_obj.topology.loads_df = loads_df.set_index('name')
edisgo_obj.topology.switches_df = switches_df.set_index('name')
edisgo_obj.topology.transformers_hvmv_df = transformers_hvmv_df.set_index('name')
edisgo_obj.topology.transformers_df = transformers_df.set_index('name')
edisgo_obj.topology.storage_units_df = storage_units_df.set_index('name')

In [146]:
# creating the mv grid
edisgo_obj.topology.mv_grid = MVGrid(id=mv_grid_id, edisgo_obj=edisgo_obj)
edisgo_obj.topology._grids = {}
edisgo_obj.topology._grids[str(edisgo_obj.topology.mv_grid)] = edisgo_obj.topology.mv_grid

#creating the lv grid
lv_grid_df =  search_str_df(buses_df,'name','LV',invert=False)
lv_grid_ids = lv_grid_df['lv_grid_id'].unique()
for lv_grid_id in lv_grid_ids:
        lv_grid = LVGrid(id=lv_grid_id, edisgo_obj=edisgo_obj)
        edisgo_obj.topology.mv_grid._lv_grids.append(lv_grid)
        edisgo_obj.topology._grids[str(lv_grid)] = lv_grid

# df = getattr(edisgo_obj.topology, "loads" + "_df")
# print (df.bus[0])
missing = df.index[~df.bus.isin(edisgo_obj.topology.buses_df.index)]
# print ("Missing: " + missing)
# print (edisgo_obj.topology.buses_df.index)

# _validate_ding0_grid_import(edisgo_obj.topology)

# get_component_timeseries(edisgo_obj=edisgo_obj, mode='worst-case')
# get_component_timeseries(edisgo_obj=edisgo_obj, mode='worst-case-feedin')


print(edisgo_obj.topology)
print ('Alles Gut')

Network topology MV4.101
Alles Gut


## importing timeseries

In [149]:
from datetime import datetime
load_profile_df = simbench_dict['LoadProfile']
res_profile = simbench_dict['RESProfile']

time_str = load_profile_df['time'][0]
time_dt = datetime.strptime(time_str, "%d.%m.%Y %H:%M")
time_pd_dt = pd.DatetimeIndex([time_dt],freq='15min')
# print (type(time_pd_dt))
# print ((time_pd_dt))
edisgo_obj.timeseries._timeindex = time_pd_dt
display(edisgo_obj.timeseries._timeindex)





DatetimeIndex(['2016-01-01'], dtype='datetime64[ns]', freq='15T')

In [148]:
# print(edisgo_obj.__dict__)
# print(edisgo_obj.topology.__dict__)
# edisgo_obj.to_graph()
# edisgo_obj.plot_mv_grid_topology(technologies=False,node_color='technology')
print(type(edisgo_obj.timeseries.__dict__))
# edisgo_obj.analyze()

<class 'dict'>


## Export to csv

In [ ]:
sb_ding0_dir = parent_dir / 'sb_ding0'
print (sb_ding0_dir)

if not sb_ding0_dir.exists():
    Path.mkdir(sb_ding0_dir)
    
print (sb_ding0_dict.keys())    
    
for key in sb_ding0_dict:
    sb_ding0_dict[key] = sb_ding0_dict[key].set_index('name')
    sb_ding0_dict[key].to_csv(str(sb_ding0_dir)+'/'+ key[:-3] +'.csv')

## Debugging code

In [152]:
get_component_timeseries(edisgo_obj=edisgo_obj, mode='worst-case-feedin')
for key in edisgo_obj.timeseries.__dict__:
    print (key)
#     display(edisgo_obj.timeseries.__dict__[key])

_timeindex
_generators_active_power
_generators_reactive_power
_loads_active_power
_loads_reactive_power
_storage_units_active_power
_storage_units_reactive_power
_charging_points_active_power
_charging_points_reactive_power
_curtailment
mode


In [153]:
for key in edisgo_obj.timeseries.__dict__:
    print (key)
    print(type(edisgo_obj.timeseries.__dict__[key]))
    display(edisgo_obj.timeseries.__dict__[key])

_timeindex
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


DatetimeIndex(['1970-01-01'], dtype='datetime64[ns]', freq='H')

_generators_active_power
<class 'pandas.core.frame.DataFrame'>


,HV1 grid at MV4.101_slack,LV1.401 SGen 1,LV1.401 SGen 2,LV1.401 SGen 4,LV1.401 SGen 3,LV2.401 SGen 1,LV2.401 SGen 2,LV2.401 SGen 3,LV2.401 SGen 4,LV2.401 SGen 5,...,MV4.101 MV SGen 1,MV4.101 MV SGen 2,MV4.101 MV SGen 3,MV4.101 MV SGen 4,MV4.101 MV SGen 5,MV4.101 MV SGen 6,MV4.101 MV SGen 7,MV4.101 MV SGen 8,MV4.101 MV SGen 9,MV4.101 MV SGen 10
1970-01-01,0.0,0.034,0.066624,0.01615,0.01955,0.01921,0.02431,0.02431,0.02346,0.00935,...,0.65,0.085,0.13,4.1,1.05,0.11,1.9,0.045,1.6,0.17
2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


_generators_reactive_power
<class 'pandas.core.frame.DataFrame'>


...

""
2016-01-01


_curtailment
<class 'pandas.core.frame.DataFrame'>


""
2016-01-01


mode
<class 'str'>


'worst-case-feedin'

,HV1 grid at MV4.101_slack,LV1.401 SGen 1,LV1.401 SGen 2,LV1.401 SGen 4,LV1.401 SGen 3,LV2.401 SGen 1,LV2.401 SGen 2,LV2.401 SGen 3,LV2.401 SGen 4,LV2.401 SGen 5,...,MV4.101 MV SGen 1,MV4.101 MV SGen 2,MV4.101 MV SGen 3,MV4.101 MV SGen 4,MV4.101 MV SGen 5,MV4.101 MV SGen 6,MV4.101 MV SGen 7,MV4.101 MV SGen 8,MV4.101 MV SGen 9,MV4.101 MV SGen 10
1970-01-01,0.0,0.034,0.066624,0.01615,0.01955,0.01921,0.02431,0.02431,0.02346,0.00935,...,0.65,0.085,0.13,4.1,1.05,0.11,1.9,0.045,1.6,0.17


In [87]:
# for key in sb_ding0_dict:
#     print ("key: " + key)
#     if 'generators' or 'loads' in key:
#         search_str(sb_ding0_dict[key],'bus','HV1 Bus 13',invert=False)
#     elif 'lines' or 'transformers' in key:
#         search_str(sb_ding0_dict[key],'bus0','HV1 Bus 13',invert=False)
#         search_str(sb_ding0_dict[key],'bus1','HV1 Bus 13',invert=False)
#     elif 'switches' in key:
#         search_str(sb_ding0_dict[key],'bus_closed','HV1 Bus 13',invert=False)
#         search_str(sb_ding0_dict[key],'bus_open','HV1 Bus 13',invert=False)

# search_str(generators_df,'name','MV4.101 MV SGen 1',invert=False)
# search_str(buses_df,'name','HV',invert=False)
# search_str(buses_df,'name','MV4.101 busbar1.1',invert=False)
# search_str(buses_df,'name','HV1 Bus 13',invert=False)
# search_str(buses_df,'name','MV4.101 Bus 7',invert=False)
# search_str(lines_df,'bus0','MV4.101 busbar1.1',invert=False)
# search_str(transformers_df,'bus0','MV4.101 Bus 7',invert=False)
# search_str(buses_df,'name','LV3.405 Bus 16',invert=False)




## Debugging section 2

In [ ]:
from edisgo.io.ding0_import import import_ding0_grid, _validate_ding0_grid_import

curr_path = Path.cwd()
parent = curr_path.parents[3]
sb_ding0_dir = parent / 'sb_ding0'
ding0_dir = parent / 'ding0_test_network'

search_str(generators_df,'name','HV1 grid at MV4.101_slack',invert=False)
search_str(buses_df,'name','HV1 Bus 13',invert=False)

edisgo_obj = EDisGo(import_timeseries=False)
print ('Hey')
import_ding0_grid(sb_ding0_dir, edisgo_obj)
print ('Hey Kern')
# _validate_ding0_grid_import(edisgo_obj.topology)



# edisgo_obj.analyze()